In [1]:
# This fetches the current poems from a private Drupal site accessed via xmlrpc

import xmlrpcsettings # this holds a few private settings
import xmlrpc.client
import re
import json
import urllib.request

assert xmlrpcsettings.ursite
assert xmlrpcsettings.inbookurl

proxy = xmlrpc.client.ServerProxy(xmlrpcsettings.ursite)


open_url = urllib.request.urlopen(xmlrpcsettings.inbookurl)
open_url = urllib.request.urlopen(xmlrpcsettings.inbookurl)
x = open_url.read().decode('utf-8')
nds = json.loads  ( x)

items = nds['Items']
inbook = []
for i in items:
    inbook.append( proxy.node.retrieve(i['node']['nid']) )
assert len(inbook)==30


In [2]:
def clean_trans(s):
    s = s.replace('--',u'—')
    s = s.replace(' -o- ','-o-')
    s = s.replace('\t','    ')# -o- ','-o-')
    return u''+s

def cap_trans(s):
    if s[0]==u'ḥ':
        s=u'Ḥ'+s[1:]
    return u''+s


In [3]:
import datetime
import os
import codecs


def generate_rst(n,poem_id,nodes): 
    '''
    generates RestructuredText string from Drupal node
    n = drupal node
    poem_id = id in book, starting at 1
    
    Model:
      
    .. title: test post
    .. slug: test-post
    .. date: 2014-09-01 03:10:12 UTC
    .. tags: 
    .. link: 
    .. description: 
    .. type: text

    '''
#    n=inbook[25]
    def get_title_trans(n):
        return u''+cap_trans(clean_trans(n['field_title_transliterated']['und'][0]['value']))
    text_trans = clean_trans(n['field_text_transliterated']['und'][0]['value'])
    
    text_trans_lines = text_trans.split('\n')
    text_trans_lines = ['| '+s for s in text_trans_lines]
    trans_title = get_title_trans(n)
    text_transliterated_rst = '\n'.join(text_trans_lines)
    
    admonition = """
.. admonition:: I Too Have Some Dreams: N. M. Rashed and Modernism in Urdu Poetry

  A translation of this Urdu poem by N. M. Rashed as well as this transliteration appears in the
  appendix of *I Too Have Some Dreams*. The transliteration is intended for
  people who can understand Urdu/Hindi or related languages. I hope to soon 
  add performances of these poems as well. 
  
  .. link_figure:: /itoohavesomedreams/
        :title: I Too Have Some Dreams Resource Page
        :class: link-figure
        :image_url: /galleries/i2havesomedreams/i2havesomedreams-small.jpg
        
"""

    utc_datetime = datetime.datetime.utcnow()
    slug = 'itoohavesomedreams/poem_'+str(poem_id)    
    time_string = utc_datetime.strftime("%Y-%m-%d %H:%M:%S")+" UTC"
    
    rst_text = u'.. title: %s\n'       % (u'§'+str(poem_id)+'. '+trans_title)
    rst_text+= u'.. slug: %s\n'        % slug
    
    rst_text+= u'.. date: %s\n'        % time_string
    rst_text+= u'.. tags: %s\n'        % 'poem itoohavesomedreams rashid'
    rst_text+= u'.. link: %s\n'        % '' # what is this?
    rst_text+= u'.. description: %s\n' % ('transliterated version of "'+trans_title+'"')
    rst_text+= u'.. type: text'+'\n'
    rst_text+= u'\n'
    rst_text+= u'\n\n'    
    rst_text+= text_transliterated_rst
    rst_text+= u'\n\n'
    add_later=u''
    if poem_id>1:
        rst_text+=u'|left arrow link|_\n'
        next_poem = nodes[poem_id-2]
        prev_title = get_title_trans(nodes[poem_id-2])
        add_later+=u"\n.. |left arrow link| replace:: :emoji:`arrow_left` §{poem_id}. {poem_title} ".format(
            poem_id=poem_id-1,
            poem_title=prev_title#u''+get_title_trans(nodes[poem_id-2]) # could adjust -1 in def
        )
        add_later+='\n.. _left arrow link: /itoohavesomedreams/poem_{poem_id}\n'.format(poem_id=poem_id-1)
    if poem_id<len(nodes):
        rst_text+='\n|right arrow link|_\n'
        add_later+=u"\n.. |right arrow link| replace::  §{poem_id}. {poem_title} :emoji:`arrow_right` ".format(
            poem_id=poem_id+1,
            poem_title=get_title_trans(nodes[poem_id])#prev_title#u''+get_title_trans(nodes[poem_id-2]) # could adjust -1 in def
        )

        add_later+='\n.. _right arrow link: /itoohavesomedreams/poem_'+str(poem_id+1)+'\n'
    rst_text+='\n\n'+add_later+admonition;
#    rst_text+= u'\n\n\u2403\n'#\u2403
#    print rst_text

    assert os.path.isdir('../itoohavesomedreams')
    filename_en = '../'+slug+'.rst'
    with codecs.open(filename_en,'w','utf-8') as f: # physical location does not matter though for slug
        f.write(rst_text)

        
def generate_ur_rst(n,poem_id,nodes): 
    '''
    generates RestructuredText string from Drupal node
    n = drupal node
    poem_id = id in book, starting at 1
    
    Model:
      
    .. title: test post
    .. slug: test-post
    .. date: 2014-09-01 03:10:12 UTC
    .. tags: 
    .. link: 
    .. description: 
    .. type: text

    '''
#    n=inbook[25]
    def get_ur_title(n):
        return n['field_title_urdu']['und'][0]['value']
    text_ur =n['field_text_urdu']['und'][0]['value']
    text_ur_lines = text_ur.split('\n')
    text_ur_lines = ['| '+s for s in text_ur_lines]
    text_ur_rst = '\n'.join(text_ur_lines)
    trans_title = cap_trans(clean_trans(n['field_title_transliterated']['und'][0]['value']))    
    ur_title = get_ur_title(n)#n['field_title_urdu']['und'][0]['value']
    utc_datetime = datetime.datetime.utcnow()
    slug = 'itoohavesomedreams/poem_'+str(poem_id)    
    time_string = utc_datetime.strftime("%Y-%m-%d %H:%M:%S")+" UTC"
    
    rst_text = u'.. title: %s\n'       % (u'§'+str(poem_id)+u'ـ '+ur_title)
    rst_text+= u'.. slug: %s\n'        % slug
    
    rst_text+= u'.. date: %s\n'        % time_string
    rst_text+= u'.. tags: %s\n'        % 'poem itoohavesomedreams rashid'
    rst_text+= u'.. link: %s\n'        % '' # what is this?
    rst_text+= u'.. description: %s\n' % ('Urdu version of "'+trans_title+'"')
    rst_text+= u'.. type: text'+'\n'
    rst_text+= u'\n'
    rst_text+= u'\n\n'    
    rst_text+= text_ur_rst+'\n\n'
#    rst_text+= u'\n\n\u2403\n'#\u2403
 #   print rst_text
    add_later=u''
    admonition=u"""
.. admonition:: I Too Have Some Dreams: N. M. Rashed and Modernism in Urdu Poetry

  یہ ن م راشد کی نظم ہے ـ اس کا انگریزی ترجمہ اور ٹرانزلٹریشن میری کتاب
  کے ضمیمہ میں مل سکتا ہےـ اردو
  پڑھنے والوں کے لئے یہ پیج پیش کیا گیا ہےـ نستعلیق میں
  دکھانے کے لئے 
  `جمیل نوری نستعلیق فانٹ`_  انسٹال کیجئے.


  .. link_figure:: /itoohavesomedreams/
        :title: I Too Have Some Dreams Resource Page
        :class: link-figure
        :image_url: /galleries/i2havesomedreams/i2havesomedreams-small.jpg
        
.. _جمیل نوری نستعلیق فانٹ: http://ur.lmgtfy.com/?q=Jameel+Noori+nastaleeq
 

"""
    if poem_id>1:
      rst_text+=u'\n|right arrow link|_\n'
      next_poem = nodes[poem_id-2]
      prev_title = get_ur_title(nodes[poem_id-2])
      add_later+=u"\n.. |right arrow link| replace:: :emoji:`arrow_right` §{poem_id}. {poem_title}  ".format(
          poem_id=poem_id-1,
          poem_title=prev_title#u''+get_title_trans(nodes[poem_id-2]) # could adjust -1 in def
      )
      add_later+=u'\n.. _right arrow link: /ur/itoohavesomedreams/poem_{poem_id}\n'.format(poem_id=poem_id-1)
    if poem_id<len(nodes):
      rst_text+=u'\n|left arrow link|_\n'
      add_later+=u"\n.. |left arrow link| replace::   §{poem_id}. {poem_title} :emoji:`arrow_left` ".format(
          poem_id=poem_id+1,
          poem_title=get_ur_title(nodes[poem_id])#prev_title#u''+get_title_trans(nodes[poem_id-2]) # could adjust -1 in def
      )
      add_later+=u'\n.. _left arrow link: /ur/itoohavesomedreams/poem_'+str(poem_id+1)+'\n'
  
    rst_text+='\n\n'+add_later+admonition;
    
        
    assert os.path.isdir('../itoohavesomedreams')
    filename_en = '../'+slug+'.ur'+'.rst' # format is : title.languageid.rst
    
    with codecs.open(filename_en,'w','utf-8') as f: # physical location does not matter though for slug
        f.write(rst_text)
        
         
    
import sys
sys.path.append('./graphparser')
import graphparser as gp
import importlib
importlib.reload(gp)

dp = gp.GraphParser('graphparser/settings/devanagari.yaml')



In [4]:
def index_link(n,n_id):
    trans_title = cap_trans(clean_trans(n['field_title_transliterated']['und'][0]['value']))
    print ('  `§'+str(n_id)+u'. '+trans_title+u' <poem_'+str(n_id)+u'/>'+'`_'+'\n')



In [5]:
dp.parse('ek din--').output#--laarins baa;g me;n').output

'एक दिन—'

In [6]:
def clean_hi_trans(s):
#    s = s.replace('--',u'—')
    s = s.replace(' -o- ','-o-')
    s = s.replace('\t','    ')# -o- ','-o-')
    s = s.replace('^','')
#    s= s.replace('-e ','-e-')
    return u''+s


def get_hi_title(n):
        t = n['field_title_markup']['und'][0]['value']
        t = t.lower()
        print(dp.parse(t).output)
        return t
def get_hi_text(n):
    t = n['body']['und'][0]['value']
    t = t.lower()
    lines = t.split('\n')
    hi_lines = [dp.parse(clean_hi_trans(l)).output for l in lines]
    hi_lines = ['| '+s for s in hi_lines]
    out = '\n'.join(hi_lines)
#    out = ['| '+s for s in out]
    return out
    

def generate_hi_rst(n,poem_id,nodes): 
    '''
    generates RestructuredText string from Drupal node
    n = drupal node
    poem_id = id in book, starting at 1
    
    Model:
      
    .. title: test post
    .. slug: test-post
    .. date: 2014-09-01 03:10:12 UTC
    .. tags: 
    .. link: 
    .. description: 
    .. type: text

    '''
#    n=inbook[25]
    def get_ur_title(n):
        return n['field_title_urdu']['und'][0]['value']
    
    def get_hi_title(n):
        t = n['field_title_markup']['und'][0]['value']
        t = t.lower()
        return dp.parse(t).output
        
    
#  #  text_hi =n['field_text_urdu']['und'][0]['value']
#   text_ur_lines = text_ur.split('\n')
#   text_ur_lines = ['| '+s for s in text_ur_lines]
    text_ur_rst = get_hi_text(n)
    
    trans_title = cap_trans(clean_trans(n['field_title_transliterated']['und'][0]['value']))    
    ur_title = get_hi_title(n)#n['field_title_urdu']['und'][0]['value']
    utc_datetime = datetime.datetime.utcnow()
    slug = 'itoohavesomedreams/poem_'+str(poem_id)    
    time_string = utc_datetime.strftime("%Y-%m-%d %H:%M:%S")+" UTC"
    
    rst_text = u'.. title: %s\n'       % (u'§'+str(poem_id)+u'ـ '+ur_title)
    rst_text+= u'.. slug: %s\n'        % slug
    
    rst_text+= u'.. date: %s\n'        % time_string
    rst_text+= u'.. tags: %s\n'        % 'poem itoohavesomedreams rashid'
    rst_text+= u'.. link: %s\n'        % '' # what is this?
    rst_text+= u'.. description: %s\n' % ('Devanagari version of "'+trans_title+'"')
    rst_text+= u'.. type: text'+'\n'
    rst_text+= u'\n'
    rst_text+= u'\n\n'    
    rst_text+= text_ur_rst+'\n\n'
    add_later=u''
    admonition=u"""
.. admonition:: I Too Have Some Dreams: N. M. Rashed and Modernism in Urdu Poetry


  .. link_figure:: /itoohavesomedreams/
        :title: I Too Have Some Dreams Resource Page
        :class: link-figure
        :image_url: /galleries/i2havesomedreams/i2havesomedreams-small.jpg
        
.. _جمیل نوری نستعلیق فانٹ: http://ur.lmgtfy.com/?q=Jameel+Noori+nastaleeq
 

""" 
    if poem_id>1:
        rst_text+=u'|left arrow link|_\n'
        next_poem = nodes[poem_id-2]
        prev_title = get_hi_title(nodes[poem_id-2])
        add_later+=u"\n.. |left arrow link| replace:: :emoji:`arrow_left` §{poem_id}. {poem_title} ".format(
            poem_id=poem_id-1,
            poem_title=prev_title#u''+get_title_trans(nodes[poem_id-2]) # could adjust -1 in def
        )
        add_later+='\n.. _left arrow link: /hi/itoohavesomedreams/poem_{poem_id}\n'.format(poem_id=poem_id-1)
    if poem_id<len(nodes):
        rst_text+='\n|right arrow link|_\n'
        add_later+=u"\n.. |right arrow link| replace::  §{poem_id}. {poem_title} :emoji:`arrow_right` ".format(
            poem_id=poem_id+1,
            poem_title=get_hi_title(nodes[poem_id])#prev_title#u''+get_title_trans(nodes[poem_id-2]) # could adjust -1 in def
        )

        add_later+='\n.. _right arrow link: /hi/itoohavesomedreams/poem_'+str(poem_id+1)+'\n'
    rst_text+='\n\n'+add_later+admonition;
    assert os.path.isdir('../itoohavesomedreams')
    filename_en = '../'+slug+'.hi'+'.rst' # format is : title.languageid.rst
    
    with codecs.open(filename_en,'w','utf-8') as f: # physical location does not matter though for slug
        f.write(rst_text)
        

In [7]:
def punjabify_hi(x):
    out=u''
    for c in x:
        if ord(c) in range(0x900, 0x97F):
            out+= chr(ord(c) + 0xA00 - 0X900)  
        else:
            out += c
    return out

print(punjabify_hi(u'है'))

ਹੈ


In [8]:
punjabify_hi(x)
def generate_pa_rst(poem_id): 
    slug = 'itoohavesomedreams/poem_'+str(poem_id) 

    filename_hi = '../'+slug+'.hi'+'.rst' # format is : title.languageid.rst
    with codecs.open(filename_hi,'r','utf-8') as f: # physical location does not matter though for slug
        text=f.read()

    new_text=punjabify_hi(text)

    filename_pa = '../'+slug+'.pa'+'.rst'
    with codecs.open(filename_pa,'w','utf-8') as f: # physical location does not matter though for slug
        f.write(new_text)


In [10]:


for n_id, n in enumerate(inbook):
    generate_rst(n,n_id+1,inbook)
    generate_ur_rst(n,n_id+1,inbook)
    generate_hi_rst(n,n_id+1,inbook)#,inbook)
    generate_pa_rst(n_id+1)#,inbook)
    index_link(n,n_id+1)
    

  `§1. Bādal (sāneṭ) <poem_1/>`_

  `§2. Ek din—lārins bāġh meñ (ek kaifiyat) <poem_2/>`_

  `§3. Sitāre (sāneṭ) <poem_3/>`_

  `§4. Vādī-e pinhāñ <poem_4/>`_

  `§5. Gunāh aur muḥabbat <poem_5/>`_

  `§6. Mukāfāt <poem_6/>`_

  `§7. Ḥuzn-e insān (aflāt̤ūnī ʿishq par ek t̤anz) <poem_7/>`_

  `§8. Ittifāqāt <poem_8/>`_

  `§9. Shāʿir-e dar-māñdah <poem_9/>`_

  `§10. Intiqām <poem_10/>`_

  `§11. Hamah ūst <poem_11/>`_

  `§12. Tel ke saudāgar <poem_12/>`_

  `§13. Mann-o-salvâ <poem_13/>`_

  `§14. Tamāshāgah-e lālah-zār <poem_14/>`_

  `§15. Namrūd kī ḳhudāʾī <poem_15/>`_

  `§16. Vuh ḥarf-e tanhā (jise tamannā-e vaṣl-e maʿnā) <poem_16/>`_

  `§17. Zindagī se ḍarte ho? <poem_17/>`_

  `§18. Dil, mire ṣaḥrā-navard-e pīr dil <poem_18/>`_

  `§19. Ek aur shahr <poem_19/>`_

  `§20. Reg-e dīrūz <poem_20/>`_

  `§21. Zamānah ḳhudā hai <poem_21/>`_

  `§22. Afsānah-e shahr <poem_22/>`_

  `§23. Yih ḳhalā pur nah huʾā <poem_23/>`_

  `§24. T̤alab ke tale <poem_24/>`_

  `§25. Gumāñ kā mumkin